In [1]:
import numpy as np 
import pandas as pd
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
dataset_folder_name = 'UTKFace/UTKFace'
TRAIN_TEST_SPLIT = 0.9
IM_WIDTH = IM_HEIGHT = 100
dataset_dict = {
    'race_id': {
        0: 'white', 
        1: 'black', 
        2: 'asian', 
        3: 'indian', 
        4: 'others'
    },
    'gender_id': {
        0: 'male',
        1: 'female'
    }
}
dataset_dict['gender_alias'] = dict((g, i) for i, g in dataset_dict['gender_id'].items())
dataset_dict['race_alias'] = dict((r, i) for i, r in dataset_dict['race_id'].items())

In [2]:
def parse_dataset(dataset_path, ext='jpg'):
    """
    Used to extract information about our dataset. It does iterate over all images and return a DataFrame with
    the data (age, gender and sex) of all files.
    """
    def parse_info_from_file(path):
        """
        Parse information from a single file
        """
        try:
            filename = os.path.split(path)[1]
            filename = os.path.splitext(filename)[0]
            age, gender, race, _ = filename.split('_')
            return int(age), dataset_dict['gender_id'][int(gender)], dataset_dict['race_id'][int(race)]
        except Exception as ex:
            return None, None, None
        
    files = glob.glob(os.path.join(dataset_path, "*.%s" % ext))
    
    records = []
    for file in files:
        info = parse_info_from_file(file)
        records.append(info)
        
    df = pd.DataFrame(records)
    df['file'] = files
    df.columns = ['age', 'gender', 'race', 'file']
    df = df.dropna()
    
    return df
df = parse_dataset(dataset_folder_name)
df.head()

,age,gender,race,file
0,100.0,male,white,UTKFace/UTKFace\100_0_0_20170112213500903.jpg....
1,100.0,male,white,UTKFace/UTKFace\100_0_0_20170112215240346.jpg....
2,100.0,female,white,UTKFace/UTKFace\100_1_0_20170110183726390.jpg....
3,100.0,female,white,UTKFace/UTKFace\100_1_0_20170112213001988.jpg....
4,100.0,female,white,UTKFace/UTKFace\100_1_0_20170112213303693.jpg....


In [3]:
from keras.utils.np_utils import to_categorical
from PIL import Image
class UtkFaceDataGenerator():
    """
    Data generator for the UTKFace dataset. This class should be used when training our Keras multi-output model.
    """
    def __init__(self, df):
        self.df = df
        
    def generate_split_indexes(self):
        p = np.random.permutation(len(self.df))
        train_up_to = int(len(self.df) * TRAIN_TEST_SPLIT)
        train_idx = p[:train_up_to]
        test_idx = p[train_up_to:]
        train_up_to = int(train_up_to * TRAIN_TEST_SPLIT)
        train_idx, valid_idx = train_idx[:train_up_to], train_idx[train_up_to:]
        
        # converts alias to id
        self.df['gender_id'] = self.df['gender'].map(lambda gender: dataset_dict['gender_alias'][gender])
        self.df['race_id'] = self.df['race'].map(lambda race: dataset_dict['race_alias'][race])
        self.max_age = self.df['age'].max()
        
        return train_idx, valid_idx, test_idx
    
    def preprocess_image(self, img_path):
        """
        Used to perform some minor preprocessing on the image before inputting into the network.
        """
        im = Image.open(img_path)
        im = im.resize((IM_WIDTH, IM_HEIGHT))
        im = np.array(im) / 255.0
        
        return im
        
    def generate_images(self, image_idx, is_training, batch_size=16):
        """
        Used to generate a batch with images when training/testing/validating our Keras model.
        """
        
        # arrays to store our batched data
        images, ages, races, genders = [], [], [], []
        while True:
            for idx in image_idx:
                person = self.df.iloc[idx]
                
                age = person['age']
                race = person['race_id']
                gender = person['gender_id']
                file = person['file']
                
                im = self.preprocess_image(file)
                
                ages.append(age / self.max_age)
                races.append(to_categorical(race, len(dataset_dict['race_id'])))
                genders.append(to_categorical(gender, len(dataset_dict['gender_id'])))
                images.append(im)
                
                # yielding condition
                if len(images) >= batch_size:
                    yield np.array(images), [np.array(ages), np.array(races), np.array(genders)]
                    images, ages, races, genders = [], [], [], []
                    
            if not is_training:
                break
                
data_generator = UtkFaceDataGenerator(df)
train_idx, valid_idx, test_idx = data_generator.generate_split_indexes() 

In [4]:
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Lambda
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
import tensorflow as tf
class UtkMultiOutputModel():
    """
    Used to generate our multi-output model. This CNN contains three branches, one for age, other for 
    sex and another for race. Each branch contains a sequence of Convolutional Layers that is defined
    on the make_default_hidden_layers method.
    """
    def make_default_hidden_layers(self, inputs):
        """
        Used to generate a default set of hidden layers. The structure used in this network is defined as:
        
        Conv2D -> BatchNormalization -> Pooling -> Dropout
        """
        x = Conv2D(16, (3, 3), padding="same")(inputs)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.25)(x)
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        return x
    def build_race_branch(self, inputs, num_races):
        """
        Used to build the race branch of our face recognition network.
        This branch is composed of three Conv -> BN -> Pool -> Dropout blocks, 
        followed by the Dense output layer.
        """
        x = self.make_default_hidden_layers(inputs)
        x = Flatten()(x)
        x = Dense(64)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(num_races)(x)
        x = Activation("softmax", name="race_output")(x)
        return x
    def build_gender_branch(self, inputs, num_genders=2):
        """
        Used to build the gender branch of our face recognition network.
        This branch is composed of three Conv -> BN -> Pool -> Dropout blocks, 
        followed by the Dense output layer.
        """
        x = Lambda(lambda c: tf.image.rgb_to_grayscale(c))(inputs)
        x = self.make_default_hidden_layers(inputs)
        x = Flatten()(x)
        x = Dense(2)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(num_genders)(x)
        x = Activation("sigmoid", name="gender_output")(x)
        return x
    def build_age_branch(self, inputs):   
        """
        Used to build the age branch of our face recognition network.
        This branch is composed of three Conv -> BN -> Pool -> Dropout blocks, 
        followed by the Dense output layer.
        """
        x = self.make_default_hidden_layers(inputs)
        x = Flatten()(x)
        x = Dense(32)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(1)(x)
        x = Activation("linear", name="age_output")(x)
        return x
    def assemble_full_model(self, width, height, num_races):
        """
        Used to assemble our multi-output model CNN.
        """
        input_shape = (height, width, 3)
        inputs = Input(shape=input_shape)
        age_branch = self.build_age_branch(inputs)
        race_branch = self.build_race_branch(inputs, num_races)
        gender_branch = self.build_gender_branch(inputs)
        model = Model(inputs=inputs,
                     outputs = [age_branch, race_branch, gender_branch],
                     name="face_net")
        return model
    
model = UtkMultiOutputModel().assemble_full_model(IM_WIDTH, IM_HEIGHT, num_races=len(dataset_dict['race_alias']))

In [5]:
foptimizer_function = 'adam'
model.compile(optimizer=foptimizer_function, 
              loss={
                  'age_output': 'mse', 
                  'race_output': 'categorical_crossentropy', 
                  'gender_output': 'binary_crossentropy'},
              loss_weights={
                  'age_output': 4., 
                  'race_output': 1.5, 
                  'gender_output': 0.1},
              metrics={
                  'age_output': 'mae', 
                  'race_output': 'accuracy',
                  'gender_output': 'accuracy'})

In [6]:
from keras.callbacks import ModelCheckpoint
batch_size = 32
valid_batch_size = 32
epochs = 64
train_gen = data_generator.generate_images(train_idx, is_training=True, batch_size=batch_size)
valid_gen = data_generator.generate_images(valid_idx, is_training=True, batch_size=valid_batch_size)
callbacks = [
    ModelCheckpoint("./model_checkpoint", monitor='val_loss')
]
history = model.fit_generator(train_gen,
                    steps_per_epoch=len(train_idx)//batch_size,
                    epochs=epochs,
                    callbacks=callbacks,
                    validation_data=valid_gen,
                    validation_steps=len(valid_idx)//valid_batch_size)

C:\Users\nhung\anaconda3\lib\site-packages\keras\engine\training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/64
600/600 [==============================] - 261s 388ms/step - loss: 7.0541 - age_output_loss: 1.1539 - race_output_loss: 1.5822 - gender_output_loss: 0.6503 - age_output_mae: 0.7477 - race_output_accuracy: 0.4604 - gender_output_accuracy: 0.6074 - val_loss: 4.4407 - val_age_output_loss: 0.7210 - val_race_output_loss: 0.9962 - val_gender_output_loss: 0.6239 - val_age_output_mae: 0.3331 - val_race_output_accuracy: 0.6700 - val_gender_output_accuracy: 0.6989
INFO:tensorflow:Assets written to: .\model_checkpoint\assets
Epoch 2/64
600/600 [==============================] - 230s 384ms/step - loss: 1.7203 - age_output_loss: 0.0611 - race_output_loss: 0.9474 - gender_output_loss: 0.5484 - age_output_mae: 0.1885 - race_output_accuracy: 0.6545 - gender_output_accuracy: 0.6796 - val_loss: 5.0173 - val_age_output_loss: 0.9477 - val_race_output_loss: 0.7900 - val_gender_output_loss: 0.4128 - val_age_output_mae: 0.3280 - val_race_output_accuracy: 0.7225 - val_gender_output_accuracy: 0.8546

600/600 [==============================] - 247s 412ms/step - loss: 1.0772 - age_output_loss: 0.0118 - race_output_loss: 0.6603 - gender_output_loss: 0.3959 - age_output_mae: 0.0824 - race_output_accuracy: 0.7663 - gender_output_accuracy: 0.7928 - val_loss: 0.9644 - val_age_output_loss: 0.0112 - val_race_output_loss: 0.5949 - val_gender_output_loss: 0.2730 - val_age_output_mae: 0.0841 - val_race_output_accuracy: 0.7940 - val_gender_output_accuracy: 0.8878
INFO:tensorflow:Assets written to: .\model_checkpoint\assets
Epoch 17/64
600/600 [==============================] - 243s 406ms/step - loss: 1.0571 - age_output_loss: 0.0116 - race_output_loss: 0.6477 - gender_output_loss: 0.3893 - age_output_mae: 0.0817 - race_output_accuracy: 0.7760 - gender_output_accuracy: 0.7932 - val_loss: 1.1255 - val_age_output_loss: 0.0577 - val_race_output_loss: 0.5792 - val_gender_output_loss: 0.2585 - val_age_output_mae: 0.1296 - val_race_output_accuracy: 0.7959 - val_gender_output_accuracy: 0.8954
INFO:tens

600/600 [==============================] - 240s 399ms/step - loss: 0.9442 - age_output_loss: 0.0097 - race_output_loss: 0.5792 - gender_output_loss: 0.3657 - age_output_mae: 0.0749 - race_output_accuracy: 0.7983 - gender_output_accuracy: 0.8033 - val_loss: 1.0773 - val_age_output_loss: 0.0388 - val_race_output_loss: 0.5977 - val_gender_output_loss: 0.2572 - val_age_output_mae: 0.1699 - val_race_output_accuracy: 0.7959 - val_gender_output_accuracy: 0.8906
INFO:tensorflow:Assets written to: .\model_checkpoint\assets
Epoch 32/64
600/600 [==============================] - 254s 423ms/step - loss: 0.9440 - age_output_loss: 0.0098 - race_output_loss: 0.5786 - gender_output_loss: 0.3703 - age_output_mae: 0.0750 - race_output_accuracy: 0.7978 - gender_output_accuracy: 0.8012 - val_loss: 1.0286 - val_age_output_loss: 0.0331 - val_race_output_loss: 0.5787 - val_gender_output_loss: 0.2807 - val_age_output_mae: 0.1567 - val_race_output_accuracy: 0.8016 - val_gender_output_accuracy: 0.8797
INFO:tens

600/600 [==============================] - 241s 402ms/step - loss: 0.8801 - age_output_loss: 0.0087 - race_output_loss: 0.5389 - gender_output_loss: 0.3695 - age_output_mae: 0.0706 - race_output_accuracy: 0.8108 - gender_output_accuracy: 0.8027 - val_loss: 0.9782 - val_age_output_loss: 0.0065 - val_race_output_loss: 0.6166 - val_gender_output_loss: 0.2718 - val_age_output_mae: 0.0590 - val_race_output_accuracy: 0.7841 - val_gender_output_accuracy: 0.8835
INFO:tensorflow:Assets written to: .\model_checkpoint\assets
Epoch 47/64
600/600 [==============================] - 244s 406ms/step - loss: 0.8505 - age_output_loss: 0.0086 - race_output_loss: 0.5201 - gender_output_loss: 0.3609 - age_output_mae: 0.0704 - race_output_accuracy: 0.8150 - gender_output_accuracy: 0.8059 - val_loss: 0.9495 - val_age_output_loss: 0.0069 - val_race_output_loss: 0.5978 - val_gender_output_loss: 0.2522 - val_age_output_mae: 0.0635 - val_race_output_accuracy: 0.7983 - val_gender_output_accuracy: 0.8911
INFO:tens

600/600 [==============================] - 256s 427ms/step - loss: 0.8148 - age_output_loss: 0.0081 - race_output_loss: 0.4975 - gender_output_loss: 0.3612 - age_output_mae: 0.0685 - race_output_accuracy: 0.8233 - gender_output_accuracy: 0.8088 - val_loss: 1.0652 - val_age_output_loss: 0.0095 - val_race_output_loss: 0.6681 - val_gender_output_loss: 0.2482 - val_age_output_mae: 0.0703 - val_race_output_accuracy: 0.7765 - val_gender_output_accuracy: 0.8963
INFO:tensorflow:Assets written to: .\model_checkpoint\assets
Epoch 62/64
600/600 [==============================] - 255s 425ms/step - loss: 0.8260 - age_output_loss: 0.0080 - race_output_loss: 0.5058 - gender_output_loss: 0.3521 - age_output_mae: 0.0683 - race_output_accuracy: 0.8199 - gender_output_accuracy: 0.8150 - val_loss: 0.9795 - val_age_output_loss: 0.0124 - val_race_output_loss: 0.6053 - val_gender_output_loss: 0.2209 - val_age_output_mae: 0.0772 - val_race_output_accuracy: 0.7955 - val_gender_output_accuracy: 0.9010
INFO:tens

In [8]:
plt.clf()
fig = go.Figure()
fig.add_trace(go.Scatter(
                    y=history.history['race_output_acc'],
                    name='Train'))
fig.add_trace(go.Scatter(
                    y=history.history['val_race_output_acc'],
                    name='Valid'))
fig.update_layout(height=500, 
                  width=700,
                  title='Accuracy for race feature',
                  xaxis_title='Epoch',
                  yaxis_title='Accuracy')
fig.show()

NameError: name 'go' is not defined

<Figure size 432x288 with 0 Axes>